In [1]:
import torch

m_id = '8d666219-bf60-42c8-a8a7-24311d9a6ec0'
save_pth = 'save/models/CIFAR10-gray.ResNet18_base.pt'
saved = torch.load(f'save/checkpoints/dir/{m_id}/checkpoint.pt.best')
m_state_dict = saved['model']

In [2]:
j_state_dict = {}

for key in m_state_dict.keys():
    if key.startswith('module.model'):
        j_state_dict[key[13:]] = m_state_dict[key]

In [3]:
from jarvis.models.resnet import resnet18

model = resnet18(in_channels=1, conv0_kernel_size=3)
model.load_state_dict(j_state_dict)

<All keys matched successfully>

In [4]:
from jarvis.vision import prepare_datasets
from torch.utils.data import DataLoader

dataset = prepare_datasets('CIFAR10', 'vision_datasets', to_grayscale=True)
loader = DataLoader(dataset, batch_size=64)

In [5]:
device = 'cuda'

model.eval().to(device)
counts = 0
for images, labels in loader:
    with torch.no_grad():
        # logits = model(((images-0.5)/0.2).to(device)).cpu()
        logits = model((images).to(device)).cpu()
        _, predicts = logits.max(dim=1)
    counts += (predicts==labels).sum()
print('acc: {:.2%}'.format(counts/len(dataset)))

acc: 92.14%


In [6]:
torch.save({
    'task': 'CIFAR10',
    'arch': 'ResNet18',
    'model': model,
    'acc': counts/len(dataset),
}, save_pth)